<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h><b>План выполнения проекта</b></h>
 1. Загрузить и изучить данные.
 2. Ответить на вопрос и обосновать решение.
    Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
 - Изменится. Приведите примеры матриц.
 - Не изменится. Как связаны параметры линейной регрессии в исходной задаче и в преобразованной?
 3. Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется.
 4. Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Предобработка:
 - Столбцы на русском языке, нужно переименовать
 - Числовой формат у столбцов "Возраст" и "Зарплата" приведем к int

In [4]:
df = df.rename(columns={'Пол':'gender','Возраст':'age','Зарплата':'salary','Члены семьи':'family_members','Страховые выплаты':'payments'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender            5000 non-null int64
age               5000 non-null float64
salary            5000 non-null float64
family_members    5000 non-null int64
payments          5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df['gender'].value_counts()

0    2505
1    2495
Name: gender, dtype: int64

In [6]:
df['age'].describe()

count    5000.000000
mean       30.952800
std         8.440807
min        18.000000
25%        24.000000
50%        30.000000
75%        37.000000
max        65.000000
Name: age, dtype: float64

In [7]:
df['age'] = df['age'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender            5000 non-null int64
age               5000 non-null int64
salary            5000 non-null float64
family_members    5000 non-null int64
payments          5000 non-null int64
dtypes: float64(1), int64(4)
memory usage: 195.4 KB


In [8]:
df['salary'].describe()

count     5000.000000
mean     39916.360000
std       9900.083569
min       5300.000000
25%      33300.000000
50%      40200.000000
75%      46600.000000
max      79000.000000
Name: salary, dtype: float64

In [9]:
df['salary'] = df['salary'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender            5000 non-null int64
age               5000 non-null int64
salary            5000 non-null int64
family_members    5000 non-null int64
payments          5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


In [10]:
df['family_members'].value_counts()

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: family_members, dtype: int64

In [11]:
df['payments'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: payments, dtype: int64

Проверим на дубликаты

In [12]:
df.duplicated().sum()

153

In [13]:
df = df.drop_duplicates()

In [14]:
df.duplicated().sum()

0

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Решим задачу, путем 

Представим новую матрицу признаков $X1$ как произведение старой $X$ на обратимую матрицу $P$:

$$
X1 = X * P
$$

Подставим значение X1 в формулу $w1$:

$$
w1 = ((X P)^T X P)^{-1} (X P)^T y
$$

Раскроем скобки $ (XP)^T $:

$$
w1 = (P^T X^T X P)^{-1} P^T X^T y
$$

"От перестановки множителей произведение не меняется":

$$
w1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно еденичной матрице ($Е$):

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Cправа получилась формула для $w$:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Подставим новое значение весов  𝑤1  в формулу для предсказаний линейной регрессии.
Предсказания модели расчитываются по формуле:

$$
a = Xw
$$

Подставим в эту формулу значения $X1$ и $w1$ для того, чтобы на их основе вычислить предсказания модели $a1$:

$$
a1 = X1w1 = XPP^{-1} w
$$

Так как $P$ по условию обратимая, то произведение $PP^{-1}$ снова равно $E$:

$$
a1 = XPP^{-1} w = X E w = X w = a
$$

Мы доказали, что предсказания $a1$ для матрицы признаков, умноженных на обратимую матрицу $P$ равны предсказаниям $a$.<br>
Следовательно, отвечая на вопрос изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу - нет, не изменится. В данном случае, параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w1$ связаны следующим образом:

$$
w1 = P^{-1} w
$$

### Вывод:
При умножении признаков на обратимую матрицу, предсказания модели не поменяется, но придется переобучать модель на новых признаках.

## Алгоритм преобразования

**Алгоритм**

Алгоритм преобразования: умножение признаков на обратимую матрицу.

**Обоснование**

Сгенерируем случайную матрицу

In [15]:
r_matrix = np.random.randint(100,size = (4,4))
r_matrix

array([[57, 28, 75, 28],
       [74, 96, 56, 18],
       [45, 46, 43,  0],
       [50, 96, 85,  2]])

Проверка матрицы на обратимость

In [16]:
matrix_inverted = np.linalg.inv(r_matrix)
matrix_inverted

array([[-0.0002378 ,  0.0030728 ,  0.0444993 , -0.02432602],
       [-0.00912733,  0.0130649 , -0.02125496,  0.01019858],
       [ 0.01001298, -0.01719212, -0.00057535,  0.01454736],
       [ 0.01850522,  0.02673009, -0.06779177,  0.00035612]])

Выделим features и target

In [17]:
features = df.drop('payments', axis = 1)
target = df['payments']

Масштабируем признаки

In [18]:
scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

Рассчитаем вектор весов

In [19]:
w = np.linalg.inv(features_scaled.T.dot(features_scaled)).dot(features_scaled.T).dot(target)

Вектор предсказаний

In [20]:
a = features_scaled @ w

Расчитаем веса для преобразованных признаков

In [21]:
features_scaled_p = features_scaled @ r_matrix

Вектор весов для преобразованных признаков

In [22]:
w1 = np.linalg.inv(features_scaled_p.T.dot(features_scaled_p)).dot(features_scaled_p.T).dot(target)

Вектор предсказаний по преобразованным признакам

In [23]:
a1 = features_scaled_p @ w1

Разность векторов

In [24]:
(a - a1).sum()

8.25962562234217e-14

Разность очень маленькая, расхождения минимальны. Можно приступать к проверке алгоритма

## Проверка алгоритма

In [26]:
model = LinearRegression(normalize = True)
model.fit(features, target)
predictions = model.predict(features)
print('MSE:{:.5}'.format(mean_squared_error(target, predictions)))
print('R2 на исходных данных:{:.5}'.format(r2_score(target, predictions)))

MSE:0.12527
R2 на исходных данных:0.4302


In [27]:
features_matrix = features_scaled @ r_matrix
model.fit(features_matrix, target)
predictions_matrix = model.predict(features_matrix)
print('MSE:{:.5}'.format(mean_squared_error(target, predictions_matrix)))
print('R2 для преобразованных  признаков:{:.5}'.format(r2_score(target, predictions_matrix)))

MSE:0.12527
R2 для преобразованных  признаков:0.4302


Значение метрик одинково, что до, что после преобразования. Данные зашифрованы, без потери качества модели.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования